importing the necessary libraries

In [1]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00


In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re

In [3]:
articles = pd.read_csv('/content/training_data.csv')
print(articles.head())

  company_name                                    job_description  \
0       Google  minimum qualifications\r\nbachelors degree or ...   
1        Apple  description\r\nas an asc you will be highly in...   
2      Netflix  its an amazing time to be joining netflix as w...   
3  Robert Half  description\r\n\r\nweb designers looking to ex...   
4    TrackFive  at trackfive weve got big goals were on a miss...   

                              position_title  description_length  \
0                           Sales Specialist                2727   
1                 Apple Solutions Consultant                 828   
2  Licensing Coordinator - Consumer Products                3205   
3                               Web Designer                2489   
4                              Web Developer                3167   

                                      model_response  
0   {\r\n  "Core Responsibilities": "Responsible ...  
1   {\r\n  "Core Responsibilities": "as an asc yo...  
2   {\r\n  

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
data = list(articles['job_description'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

The Doc2vec model is initialized as follows

In [6]:
model = Doc2Vec(vector_size = 50,
min_count = 10,
epochs = 50
)

vocabulary building

In [9]:
model.build_vocab(tagged_data)
k = model.wv.key_to_index
print(len(k))

3564


In [10]:
model.train(tagged_data,
total_examples = model.corpus_count,
epochs = model.epochs)
model.save('doc2vec.model')
print("Model saved")

Model saved


In [34]:
resume_path = '/content/717821i239-Ramasamy_V.pdf'
resume = ''
# Open the PDF file
with open(resume_path, 'rb') as file:
    # Create a PdfReader object
    pdfReader = PyPDF2.PdfReader(file)

    # Iterate through each page
    for i in range(len(pdfReader.pages)):
        # Access the page using indexing
        pageObj = pdfReader.pages[i]

        # Extract text from the page and add it to the resume string
        resume += pageObj.extract_text()

In [35]:
resume = resume.lower()
resume = re.sub('[^a-z]', ' ', resume)

In [36]:
def extract_data(url):
  list1 = []
  count = 0
  resp = requests.get(url)
  if resp.status_code == 200:
      soup = BeautifulSoup(resp.text,'html.parser')
      l = soup.find(class_ = 'av-company-description-page mb-2')
      web = ''.join([i.text for i in l.find_all(['p', 'li'])])
      list1.append(web)
      return web
  else:
      print("Error")
jd_links = ['https://datahack.analyticsvidhya.com/jobathon/cropin/data-scientist-85']

jd_df = pd.DataFrame(columns = ['links', 'data'])
jd_df['links'] = jd_links

#Extracting the JD data from the links:
for i in range(len(jd_df)):
  jd_df['data'][i] = extract_data(jd_df['links'][i])

In [37]:
#Converting the text into lower case
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : str.lower(x))

#Removing the punctuations from the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : " ".join(re.findall('[\w]+',x))
)
#Removing the numerics present in the text
jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : re.sub(r'\d+','',x))

In [38]:
from numpy.linalg import norm


In [39]:
model = Doc2Vec.load('doc2vec.model')
v1 = model.infer_vector(resume.split())
v2 = model.infer_vector(jd_df['data'][0].split())
cosine_similarity = (np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(cosine_similarity, 3))

0.513
